In [1]:
from utils_c1 import *
from Model_c1 import *
from dataset_c1 import *
from train_c1 import *
from predict_c1 import *

In [2]:
x = np.random.rand(5,32,32,2)

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

In [4]:
dataset = CustomDataset(x, transform = transform,mode = 'test')

In [5]:
dataloader = DataLoader(dataset, batch_size = 1, shuffle = True)

In [6]:
# Since Kfold training has been performed we have ensemble of kfold models
k_folds = 5

outputs = []
models = []
predicts = []
for fold in range(k_folds):
    DEVICE = torch.device("cuda")
    model = Resnet_Ensemble(2, 15, BasicBlock, CBAM, resnet = ResNet)
    model.load_state_dict(torch.load(f'/DATA/Shashank/GSOC 2024/Common Task -1/Common Task 1/best_model_{fold}.pth'))
    model.eval()
    model = model.to('cuda')
    predict = Predict(model)
    predicts.append(predict)



for batch in dataloader:
    op = []
    img = batch['image'].to('cuda').float()
    for fold in range(k_folds):
        op.append(predicts[fold].predict(img).cpu().detach().numpy())
    op = np.array(op)
    outputs.append(op.mean())

# outputs.append(np.array(op))

In [7]:
outputs

[0.4, 0.4, 0.4, 0.4, 0.4]

In [8]:
outputs = np.array(outputs)

In [9]:
outputs.shape

(5,)

In [11]:
outputs = outputs.reshape((outputs.shape[0], 1))

In [12]:
outputs

array([[0.4],
       [0.4],
       [0.4],
       [0.4],
       [0.4]], dtype=float32)